In [ ]:
from pathlib import Path
import tensorflow as tf
import keras 

from keras import layers, models


print("GPU available:", tf.config.list_physical_devices('GPU'))

### Workdir

In [ ]:
data_dir = Path("work_dataset/")

### Check class mapping

In [ ]:
# TODO: sprawdzić wszystkie parametry w tym wywołaniu
base_model = keras.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(480, 480, 3),
    pooling="avg",
    # classes=1000,
    # classifier_activation="softmax",
    include_preprocessing=True,
    name="efficientnetv2-l",
)
print(base_model.input_shape)

In [ ]:
model = models.Sequential([
    base_model,
    # layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')  # sigmoid dla binary classification
])

In [ ]:
base_model.trainable = False

In [ ]:
batch_size = 256
base_lr = 1e-3
lr = base_lr * (batch_size / 32)  # => 0.004

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
lr


In [ ]:
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # 'binary_...' lub 'categorical_crossentropy' dla wieloklasowej
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# TODO: sprawdzić wszystkie parametry w tym wywołaniu

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode="categorical",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=(480, 480),
    batch_size=256,
    crop_to_aspect_ratio=True,
)

In [ ]:
train_ds.class_names

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
# Największe accuracy na danych treningowych
max_acc = max(history.history['accuracy'])

# Najmniejszy loss na danych treningowych
min_loss = min(history.history['loss'])

# Największe accuracy na danych walidacyjnych
max_val_acc = max(history.history['val_accuracy'])

# Najmniejszy loss na danych walidacyjnych
min_val_loss = min(history.history['val_loss'])

In [ ]:
best_epoch_acc = history.history['accuracy'].index(max_acc) + 1
best_epoch_loss = history.history['loss'].index(min_loss) + 1

In [ ]:
max_val_acc

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Załaduj obraz — dostosuj ścieżkę i rozmiar do wymaganego przez model
img = load_img('work_dataset/wżery na lakierze/12673_photo_(11)..jpg', target_size=(480, 480)) 

# Zamień obraz na tablicę NumPy
img_array = img_to_array(img)

# Normalizacja (jeśli była stosowana podczas treningu)
img_array = img_array / 255.0

# Dodaj dodatkowy wymiar (batch size = 1)
img_array = np.expand_dims(img_array, axis=0)  # kształt: (1, 150, 150, 3)

In [ ]:
predictions = model.predict(img_array)

In [ ]:
predictions

In [ ]:
import numpy as np

predicted_class = train_ds.class_names[np.argmax(predictions[0])]
print(f"Przewidywana klasa: {predicted_class}")